In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.datasets import load_iris
iris_data = load_iris()

In [3]:
type(iris_data)

sklearn.utils._bunch.Bunch

In [4]:
iris_df = pd.DataFrame(data = iris_data['data'], columns = iris_data['feature_names'])
iris_df['Target'] = iris_data['target']
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
set(iris_df['Target'])

{0, 1, 2}

# Etapas da crição do modelo

- Verificar se existem dados nulos
- Separar os dados em treino e teste
- Normalizar os dados
- Treinar o modelo (Random Forest)
- Validar o modelo (classification report)

In [7]:
# verificar se possui dados nulos
nulos_iris_df = iris_df.isnull().sum()
nulos_iris_df

,0
sepal length (cm),0
sepal width (cm),0
petal length (cm),0
petal width (cm),0
Target,0


In [8]:
from sklearn.model_selection import train_test_split

# Separando em treino e teste

X = iris_df.drop('Target', axis=1) # Variáveis independentes
y = iris_df['Target']

# Separar os dados em treino e teste (80% treino e 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
X_train.shape

(105, 4)

In [10]:
X_test.shape

(45, 4)

In [11]:
# normalizando os dados de treino e teste
from sklearn.preprocessing import MinMaxScaler

# Criar um objeto MinMaxScaler
scaler = MinMaxScaler()

# Ajustar o scaler aos dados de treino e transformá-los
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de teste com base na normalização do treino
X_test_scaled = scaler.transform(X_test)

In [12]:
# Treinamento do modelo

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Criar o modelo RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)

# Treinar o modelo com os dados de treino normalizados
rf_model.fit(X_train_scaled, y_train)

# Fazer previsões com os dados de teste normalizados
y_pred = rf_model.predict(X_test_scaled)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Random Forest: {accuracy * 100:.2f}%")

Acurácia do modelo Random Forest: 100.00%


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Definir o modelo base
rf_model = RandomForestClassifier(random_state=42)

# Definir a grade de parâmetros que serão testados
param_grid = {
    'n_estimators': [100, 200, 300],            # Número de árvores na floresta
    'max_depth': [10, 20, None],                # Profundidade máxima da árvore
    'min_samples_split': [2, 5, 10],            # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 2, 4],              # Número mínimo de amostras em uma folha
    'max_features': ['sqrt', 'log2', None]      # Número de features a serem consideradas
}

# Configurar o GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Treinar o modelo com os dados de treino normalizados
grid_search.fit(X_train_scaled, y_train)

# Verificar os melhores parâmetros encontrados
print(f"Melhores parâmetros: {grid_search.best_params_}")

# Fazer previsões com os dados de teste usando o melhor modelo
best_rf_model = grid_search.best_estimator_
y_pred = best_rf_model.predict(X_test_scaled)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Random Forest otimizado: {accuracy * 100:.2f}%")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Melhores parâmetros: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Acurácia do modelo Random Forest otimizado: 100.00%


In [14]:
from sklearn.metrics import classification_report

# Fazer previsões com os dados de teste usando o melhor modelo encontrado pelo Grid Search
y_pred = grid_search.best_estimator_.predict(X_test_scaled)

# Gerar o classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Criar o modelo de Árvore de Decisão com uma profundidade máxima limitada
tree_model = DecisionTreeClassifier(max_depth=2, random_state=42)

# Treinar o modelo com os dados de treino normalizados
tree_model.fit(X_train_scaled, y_train)

# Fazer previsões com os dados de teste
y_pred = tree_model.predict(X_test_scaled)

# Avaliar a acurácia do modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Decision Tree: {accuracy * 100:.2f}%")

# Gerar o classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Acurácia do modelo Decision Tree: 97.78%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.98        45
   macro avg       0.98      0.97      0.97        45
weighted avg       0.98      0.98      0.98        45



In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score

# Criar o modelo de Árvore de Decisão com parâmetros ajustados
tree_model = DecisionTreeClassifier(
    max_depth=2,                 # Limitar a profundidade máxima
    min_samples_split=10,        # Mínimo de amostras necessárias para dividir um nó
    min_samples_leaf=5,          # Mínimo de amostras em cada folha
    max_leaf_nodes=20,           # Limitar o número máximo de nós folha
    max_features='sqrt',         # Usar a raiz quadrada do número total de features
    random_state=42
)

# Treinar o modelo com os dados de treino
tree_model.fit(X_train_scaled, y_train)

# Fazer previsões com os dados de teste
y_pred = tree_model.predict(X_test_scaled)

# Avaliar o desempenho
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo Decision Tree ajustado: {accuracy * 100:.2f}%")

# Gerar o classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Acurácia do modelo Decision Tree ajustado: 100.00%
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45

